<a href="https://colab.research.google.com/github/Biruk-gebru/Causal-Gene-Prediction-and-Validation-with-LLMs/blob/main/Module1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install transformers datasets peft accelerate bitsandbytes


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "microsoft/BioGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],  # depends on model arch
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 296,450 || all params: 108,608,260 || trainable%: 0.2730


In [16]:
from datasets import load_dataset

dataset = load_dataset("pubmed_qa", "pqa_labeled")


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
dataset["train"][0]


{'pubid': 21645374,
 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
   'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), ce

In [25]:
def preprocess(example):
    question = example["question"]
    context_paragraphs = example["context"]["contexts"]
    context = " ".join(context_paragraphs)
    text = question + " " + context
    tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = 1 if example["final_decision"].strip().lower() == "yes" else 0
    return tokenized
encoded_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=10,
    output_dir="./results",
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    tokenizer=tokenizer
)

trainer.train()



<ipython-input-26-8b0089d7db4b>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: birukgebru83 (birukgebru83-addis-ababa-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.686800
20,0.715000
30,0.717900
40,0.697700
50,0.692400
60,0.703900
70,0.708600
80,0.707400
90,0.685500
100,0.709200


TrainOutput(global_step=375, training_loss=0.6959552408854167, metrics={'train_runtime': 506.5208, 'train_samples_per_second': 5.923, 'train_steps_per_second': 0.74, 'total_flos': 198016312320000.0, 'train_loss': 0.6959552408854167, 'epoch': 3.0})

In [27]:
model.save_pretrained("./biomed_stage1_lora")
tokenizer.save_pretrained("./biomed_stage1_lora")


('./biomed_stage1_lora/tokenizer_config.json',
 './biomed_stage1_lora/special_tokens_map.json',
 './biomed_stage1_lora/vocab.txt',
 './biomed_stage1_lora/added_tokens.json',
 './biomed_stage1_lora/tokenizer.json')

In [28]:
import pandas as pd

data = [
    {"text": "BRCA1 mutation causes breast cancer.", "label": 1},
    {"text": "TP53 deletion results in uncontrolled cell growth.", "label": 1},
    {"text": "Expression of GAPDH is correlated with glucose levels.", "label": 0},
    {"text": "MYC overexpression may be associated with lymphoma.", "label": 0},
    {"text": "CFTR mutation causes cystic fibrosis.", "label": 1},
    {"text": "Smoking affects gene expression of CYP1A1.", "label": 0},
]

df = pd.DataFrame(data)
df.to_csv("fewshot_trait_gene.csv", index=False)


In [29]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="fewshot_trait_gene.csv")

def preprocess_fewshot(example):
    tokenized = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = int(example["label"])
    return tokenized

encoded_fewshot = dataset.map(preprocess_fewshot, remove_columns=dataset["train"].column_names)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [30]:
from transformers import TrainingArguments

training_args_stage2 = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=10,  # more passes for few-shot
    learning_rate=5e-5,
    output_dir="./results_stage2",
    fp16=True,
    remove_unused_columns=False,
    logging_steps=2
)

trainer_stage2 = Trainer(
    model=model,  # continue from Stage 1 LoRA model
    args=training_args_stage2,
    train_dataset=encoded_fewshot["train"],
    tokenizer=tokenizer
)

trainer_stage2.train()


<ipython-input-30-1586d0641260>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_stage2 = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
2,0.770200
4,0.780900
6,0.618200
8,0.703300
10,0.657000
12,0.721600
14,0.731500
16,0.675800
18,0.742000
20,0.769900


TrainOutput(global_step=20, training_loss=0.717047119140625, metrics={'train_runtime': 3.3667, 'train_samples_per_second': 17.822, 'train_steps_per_second': 5.941, 'total_flos': 3960326246400.0, 'train_loss': 0.717047119140625, 'epoch': 10.0})

In [31]:
model.save_pretrained("./biomed_stage2_causal")
tokenizer.save_pretrained("./biomed_stage2_causal")


('./biomed_stage2_causal/tokenizer_config.json',
 './biomed_stage2_causal/special_tokens_map.json',
 './biomed_stage2_causal/vocab.txt',
 './biomed_stage2_causal/added_tokens.json',
 './biomed_stage2_causal/tokenizer.json')

In [32]:
!pip install sentence-transformers


In [33]:
import pandas as pd

contrastive_data = [
    {"anchor": "BRCA1", "positive": "breast cancer", "negative": "hypertension"},
    {"anchor": "CFTR", "positive": "cystic fibrosis", "negative": "asthma"},
    {"anchor": "TP53", "positive": "tumor suppression", "negative": "inflammation"},
    {"anchor": "APP", "positive": "Alzheimer's disease", "negative": "leukemia"}
]

df_contrastive = pd.DataFrame(contrastive_data)
df_contrastive.to_csv("gene_trait_triplets.csv", index=False)


In [34]:
from sentence_transformers import InputExample
from datasets import load_dataset

dataset = load_dataset("csv", data_files="gene_trait_triplets.csv")["train"]

examples = [
    InputExample(texts=[ex["anchor"], ex["positive"], ex["negative"]])
    for ex in dataset
]


Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
from sentence_transformers import SentenceTransformer, losses, models
from torch.utils.data import DataLoader

# Use a base sentence embedding model (or wrap your fine-tuned PEFT model)
biobert = models.Transformer("emilyalsentzer/Bio_ClinicalBERT")  # or your stage2 model path
pooling = models.Pooling(biobert.get_word_embedding_dimension(), pooling_mode='mean')

model = SentenceTransformer(modules=[biobert, pooling])


In [36]:
train_dataloader = DataLoader(examples, batch_size=4, shuffle=True)
train_loss = losses.TripletLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    warmup_steps=10,
    output_path="./biomed_stage3_contrastive"
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [40]:
from sentence_transformers import SentenceTransformer, util

# Load your Stage 3 model
model = SentenceTransformer("./biomed_stage3_contrastive")

# Sample triplet
anchor = "BRCA1"
positive = "breast cancer"
negative = "hypertension"

# Encode
embeddings = model.encode(["breast cancer", "BRCA1", "hypertension"], convert_to_tensor=True)
cos_pos = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
cos_neg = util.pytorch_cos_sim(embeddings[0], embeddings[2]).item()


print(f"Cosine Similarity (Anchor ↔ Positive): {cos_pos:.4f}")
print(f"Cosine Similarity (Anchor ↔ Negative): {cos_neg:.4f}")


Cosine Similarity (Anchor ↔ Positive): 0.6567
Cosine Similarity (Anchor ↔ Negative): 0.7956


In [42]:
# Save was already handled if you set `output_path` during training
# But you can save again explicitly:
model.save("./biomed_stage3_contrastive")


In [43]:
model.save("/content/drive/MyDrive/biomed_stage3_contrastive")
